# Cursus: Automatic SageMaker (MODS) Pipeline Compiler

The main contribution of this work is **Cursus**, a **compiler** that automatically generate **[MODS (Model Training Workflow Operation and Development System) Pipeline](https://w.amazon.com/bin/view/CMLS/Overview/MODS/)** base on two set of user inputs
* The **Pipeline DAG (Directed Acylic Graph)**, which describe pipeline as a graph
* The **Unified Config JSON**, which provides a central hub to extract all user inputs and their associated step information
    * Run [demo_config](./demo_config.ipynb) first to generate the Unified Config JSON
    * The config json will be saved in `./pipeling_config/xxx/` folder

![mods_pipeline_train_eval_calib](./demo/mods_pipeline_train_eval_calib.png)


In [1]:
import os
import json
import pandas as pd
import pickle
import sys
import subprocess
from datetime import datetime

from pathlib import Path

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from pydantic import BaseModel, Field, model_validator, field_validator
from typing import List, Optional, Dict, Any, Type, Union, Tuple


In [3]:
from collections import (
    defaultdict,
    deque
)

In [4]:
import logging

In [5]:
logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
logger = logging.getLogger(__name__)


## Environment Setup

In [6]:
from sagemaker import Session

2025-11-08 05:08:10,890 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
from sagemaker.workflow.pipeline_context import PipelineSession

In [8]:
bucket_name='buyer-seller-messaging-reversal'

In [9]:
pipeline_session = PipelineSession(default_bucket=bucket_name) # IMPORTANT now the session uses the generated sagemaker_config

2025-11-08 05:08:11,249 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [10]:
role=PipelineSession().get_caller_identity_arn()
role

2025-11-08 05:08:11,511 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


'arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default'

In [11]:
from pathlib import Path
import sys

# Get parent directory of current notebook
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)  
    print(f"add project root {project_root} into system")

add project root /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines into system


## Basic Information

In [12]:
region_list = [
    'NA',
    'EU',
    'FE'
]

In [13]:
region_selection = 0

In [14]:
region = region_list[region_selection]
region

'NA'

In [15]:
MODEL_CLASS='pytorch'

In [16]:
service_name="BuyerAbuseRnR"

#### Config and Hyperparameter Information

In [17]:
current_dir = Path.cwd()
config_dir = Path(current_dir) / 'pipeline_config'
print(config_dir)

/home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config


In [18]:
#hyparam_filename = f'hyperparameters_{region}_{MODEL_CLASS}.json' #'hyperparameters.json'

In [19]:
pipeline_config_name = f'config.json'  #f'config_{region}.json'
pipeline_config_name

'config.json'

In [20]:
config_path = config_dir / pipeline_config_name

In [21]:
config_path

PosixPath('/home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json')

## Pipeline Imports

In [22]:
from enum import Enum
from pydantic import BaseModel

## [Optional]: Test Config Load Functionality

Please skip this section if you are not concern about the config information loaded

### Hyperparameters

In [23]:
#from cursus.steps.hyperparams.hyperparameters_xgboost import XGBoostModelHyperparameters

In [24]:
#hyparam_path = config_dir / hyparam_filename
#with open(hyparam_path, 'r') as file:
#    hyperparam_dict = json.load(file)

In [25]:
#hyperparams = XGBoostModelHyperparameters(**hyperparam_dict)

In [26]:
#hyperparams.num_classes

In [27]:
#hyperparams.is_binary

### Import Configs

In [28]:
from cursus.core.base.config_base import BasePipelineConfig

2025-11-08 05:08:12,158 - WARNING - Could not import constants from mods_workflow_core, using local definitions


In [29]:
#from cursus.steps.configs.config_cradle_data_loading_step import (CradleDataLoadingConfig,
#                                                    MdsDataSourceConfig,
#                                                    EdxDataSourceConfig,
#                                                    DataSourceConfig,
#                                                    DataSourcesSpecificationConfig,
#                                                    JobSplitOptionsConfig,
#                                                    TransformSpecificationConfig,
#                                                    OutputSpecificationConfig,
#                                                    CradleJobSpecificationConfig
#                                                   )

In [30]:
from cursus.steps.configs.config_dummy_data_loading_step import DummyDataLoadingConfig
from cursus.steps.configs.config_tabular_preprocessing_step import TabularPreprocessingConfig
from cursus.steps.configs.config_bedrock_prompt_template_generation_step import BedrockPromptTemplateGenerationConfig
from cursus.steps.configs.config_bedrock_batch_processing_step import BedrockBatchProcessingConfig

### Load Config

In [31]:
from cursus.steps.configs.utils import serialize_config, merge_and_save_configs, load_configs, verify_configs

In [32]:
CONFIG_CLASSES = {
        'DummyDataLoadingConfig':                     DummyDataLoadingConfig,
        'BedrockPromptTemplateGenerationConfig':      BedrockPromptTemplateGenerationConfig,
        'BedrockBatchProcessingConfig':               BedrockBatchProcessingConfig,
        'TabularPreprocessingConfig':                 TabularPreprocessingConfig,
    }

In [33]:
config_path

PosixPath('/home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json')

In [34]:
# Load configs
loaded_configs = load_configs(config_path, CONFIG_CLASSES)

2025-11-08 05:08:12,416 - INFO - Loading configs from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-08 05:08:12,416 - INFO - Loading configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-08 05:08:12,418 - WARNING - Could not find class InstructionConfig
2025-11-08 05:08:12,418 - WARNING - Could not find class OutputFormatConfig
2025-11-08 05:08:12,418 - WARNING - Could not find class SystemPromptConfig
2025-11-08 05:08:12,419 - INFO - Successfully loaded configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-08 05:08:12,419 - INFO - Successfully loaded configs from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json with 4 specific c

In [35]:
loaded_configs

{'BedrockBatchProcessing_training': BedrockBatchProcessingConfig(author='lukexie', bucket='buyer-seller-messaging-reversal', role='arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default', region='NA', service_name='BuyerAbuseRnR', pipeline_version='0.0.1', model_class='pytorch', current_date='2025-11-08', framework_version='2.1.0', py_version='py310', source_dir='docker', enable_caching=False, use_secure_pypi=False, project_root_folder='rnr_pytorch_bedrock', processing_instance_count=1, processing_volume_size=500, processing_instance_type_large='ml.m5.12xlarge', processing_instance_type_small='ml.m5.4xlarge', use_large_processing_instance=False, processing_source_dir='docker/scripts', processing_entry_point='bedrock_batch_processing.py', processing_script_arguments=None, processing_framework_version='1.2-1', bedrock_batch_role_arn='arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default', job_type='training', bedrock_primary_model_id='anthropic.claude-sonnet-

In [36]:
len(loaded_configs)

4

In [37]:
[str(k) for k in loaded_configs.keys()]

['BedrockBatchProcessing_training',
 'BedrockPromptTemplateGeneration',
 'DummyDataLoading_training',
 'TabularPreprocessing_training']

In [38]:
first_config = next(iter(loaded_configs.values()))

In [39]:
PIPELINE_VERSION = first_config.pipeline_version

In [40]:
PIPELINE_DESCRIPTION = first_config.pipeline_description

In [41]:
PIPELINE_NAME = first_config.pipeline_name

## Parameter Setup

In [42]:
import boto3
from sagemaker.workflow.pipeline_context import PipelineSession

# Initialize boto3 clients
ec2_client = boto3.client('ec2')
kms_client = boto3.client('kms')
sts_client = boto3.client('sts')

# Get account and region info
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

2025-11-08 05:08:12,632 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


### Find VPC Subnet - Get default VPC subnets or list all

In [43]:
response = ec2_client.describe_subnets(
    Filters=[{'Name': 'default-for-az', 'Values': ['true']}]
)
vpc_subnet_id = response['Subnets'][0]['SubnetId'] if response['Subnets'] else None

# OR list all subnets and choose one
#all_subnets = ec2_client.describe_subnets()
#for subnet in all_subnets['Subnets']:
#    print(f"Subnet ID: {subnet['SubnetId']}, VPC: {subnet['VpcId']}, AZ: {subnet['AvailabilityZone']}")


### Find Security Group - Get default or list all

In [44]:
response = ec2_client.describe_security_groups(
    Filters=[{'Name': 'group-name', 'Values': ['default']}]
)
security_group_id = response['SecurityGroups'][0]['GroupId'] if response['SecurityGroups'] else None

# OR list all security groups
#all_sgs = ec2_client.describe_security_groups()
#for sg in all_sgs['SecurityGroups']:
#    print(f"SG ID: {sg['GroupId']}, Name: {sg['GroupName']}, VPC: {sg.get('VpcId')}")

### Find KMS Key - List KMS keys for SageMaker

In [45]:
response = kms_client.list_aliases()
for alias in response['Aliases']:
    if 'sagemaker' in alias['AliasName'].lower():
        print(f"KMS Alias: {alias['AliasName']}, Key ID: {alias.get('TargetKeyId')}")

# OR get account's default KMS key ARN
kms_key_id = f"arn:aws:kms:{region}:{account_id}:alias/aws/sagemaker"


In [46]:
print(f"\nFound values:")
print(f"VPC Subnet: {vpc_subnet_id}")
print(f"Security Group: {security_group_id}")
print(f"KMS Key: {kms_key_id}")


Found values:
VPC Subnet: subnet-45db3e4b
Security Group: sg-e116c4be
KMS Key: arn:aws:kms:us-east-1:178936618742:alias/aws/sagemaker


### Execution Id

In [47]:
execution_id = datetime.now().strftime("%Y%m%d%H%M%S")

### Define Parameter String

In [48]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import ParameterString


In [49]:
# Predefined Pipeline Parameters
PIPELINE_EXECUTION_TEMP_DIR = ParameterString(name="EXECUTION_S3_PREFIX", default_value=f"s3://{bucket_name}/pipeline/{PIPELINE_NAME}/{execution_id}")
KMS_ENCRYPTION_KEY_PARAM = ParameterString(name="KMS_ENCRYPTION_KEY_PARAM", default_value=kms_key_id)
VPC_SUBNET = ParameterString(
    name="VPC_SUBNET",
    default_value=vpc_subnet_id
)  # TODO: test if we can replace it with multiple subnets
SECURITY_GROUP_ID = ParameterString(name="SECURITY_GROUP_ID", default_value=security_group_id)
PROCESSING_JOB_SHARED_NETWORK_CONFIG = NetworkConfig(
    enable_network_isolation=False,
    security_group_ids=[SECURITY_GROUP_ID],
    subnets=[VPC_SUBNET],
    encrypt_inter_container_traffic=True,
)

## Import Packages

In [50]:
from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional, Type
from pathlib import Path
import logging
import os
import importlib

In [51]:
import sagemaker
from sagemaker import Session
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.parameters import Parameter
from sagemaker.workflow.properties import Properties
from sagemaker.workflow.pipeline_context import PipelineSession # Crucial import

## Demo: An End-to-End Pipeline based on PipelineDAG Compiler
Let us use the following simpler DAG (without registration as example)


In this demo there are several user input
* the **Unified JSON file** in `config_path`
* the **Registry Manager**: an object that handles the map between step logical name to `step.properties`
* the **Dependency Resolver**: an object than handles the *automatic dependency resolution* between steps
* the other fields
    * `sagemaker_session`: pipelne session
    * `role`: IAM Role
    * `notebook_root`: track the root path 


In this pipeline template, we inherit from base class `PipelineTemplateBase`. 

The **major tasks** are
* *`Config` Classes Import*
* *Configuration Validation*
* *Step Builder Retrieval and Step Builder Map Creation*
* *Configuration Map Creation*
* **Pipeline DAG Generation**: ideally, user should create this DAG and use it as input
* **Automatic Pipeline Assemble**: Call `pipeline_assembler`


### DAG to Template Compiler

In [52]:
from cursus.api.dag.base_dag import PipelineDAG
from cursus.core.compiler.dag_compiler import compile_dag_to_pipeline, PipelineDAGCompiler
from cursus.core.compiler.validation import ConversionReport
from cursus.steps.configs.utils import load_configs

In [53]:
def create_bedrock_batch_data_processing_dag() -> PipelineDAG:
    """
    Create a DAG for Bedrock Batch data processing pipeline.

    This DAG represents the simplest possible workflow that includes
    cost-efficient Bedrock batch LLM enhancement for pure data processing
    without any training, calibration, packaging, registration, or evaluation steps.
    Perfect for data enhancement and annotation workflows.

    Returns:
        PipelineDAG: The directed acyclic graph for the pipeline
    """
    dag = PipelineDAG()

    # Add minimal data processing nodes with Bedrock batch enhancement
    dag.add_node("DummyDataLoading_training")  # Dummy data load
    dag.add_node("TabularPreprocessing_training")  # Tabular preprocessing
    dag.add_node("BedrockPromptTemplateGeneration")  # Bedrock prompt template generation
    dag.add_node("BedrockBatchProcessing_training")  # Bedrock batch processing step

    # Simple data processing flow with Bedrock batch enhancement
    dag.add_edge("DummyDataLoading_training", "TabularPreprocessing_training")

    # Bedrock batch processing flow - two inputs to BedrockBatchProcessing
    dag.add_edge("TabularPreprocessing_training", "BedrockBatchProcessing_training")  # Data input
    dag.add_edge("BedrockPromptTemplateGeneration", "BedrockBatchProcessing_training")  # Template input

    logger.info(
        f"Created Bedrock Batch data processing DAG with {len(dag.nodes)} nodes and {len(dag.edges)} edges"
    )
    return dag

In [54]:
dag = create_bedrock_batch_data_processing_dag()

2025-11-08 05:08:13,426 - INFO - Added node: DummyDataLoading_training
2025-11-08 05:08:13,427 - INFO - Added node: TabularPreprocessing_training
2025-11-08 05:08:13,427 - INFO - Added node: BedrockPromptTemplateGeneration
2025-11-08 05:08:13,427 - INFO - Added node: BedrockBatchProcessing_training
2025-11-08 05:08:13,428 - INFO - Added edge: DummyDataLoading_training -> TabularPreprocessing_training
2025-11-08 05:08:13,428 - INFO - Added edge: TabularPreprocessing_training -> BedrockBatchProcessing_training
2025-11-08 05:08:13,428 - INFO - Added edge: BedrockPromptTemplateGeneration -> BedrockBatchProcessing_training
2025-11-08 05:08:13,429 - INFO - Created Bedrock Batch data processing DAG with 4 nodes and 3 edges


In [55]:
pipeline_parameters = [
    PIPELINE_EXECUTION_TEMP_DIR,
    KMS_ENCRYPTION_KEY_PARAM,
    SECURITY_GROUP_ID,
    VPC_SUBNET,
]

In [56]:
dag_compiler = PipelineDAGCompiler(
    config_path=config_path,
    sagemaker_session=pipeline_session,
    role=role,
    pipeline_parameters=pipeline_parameters
)

2025-11-08 05:08:13,438 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-08 05:08:13,439 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-11-08 05:08:13,439 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-11-08 05:08:13,440 - INFO - ✅ Registry info loaded: 34 steps
2025-11-08 05:08:13,440 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-11-08 05:08:13,441 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-08 05:08:13,441 - INFO - 🔍 ScriptAutoDiscovery.__init__ - workspace_dirs: []
2025-11-08 05:08:13,441 - INFO - 🔍 ScriptAutoDiscovery.__init__ - priority_workspace_dir: None
2025-11-08 05:08:13,442 - INFO - ✅ Registry info loaded: 34 steps
2025-11-08 05:08:13,442 - INFO - 🎉 ScriptAutoDiscovery initialization com

### Create a Pipeline

#### DAG Validation and Preview of Config Resolution

In [57]:
preview_only = False

In [58]:
if preview_only:
    preview = dag_compiler.preview_resolution(dag)
    logger.info("DAG node resolution preview:")
    for node, config_type in preview.node_config_map.items():
        confidence = preview.resolution_confidence.get(node, 0.0)
        logger.info(f"  {node} → {config_type} (confidence: {confidence:.2f})")
        
    if preview.recommendations:
        logger.info("Recommendations:")
        for recommendation in preview.recommendations:
            logger.info(f"  - {recommendation}")
        
    validation = dag_compiler.validate_dag_compatibility(dag)
    logger.info(f"DAG validation: {'VALID' if validation.is_valid else 'INVALID'}")
    if not validation.is_valid:
        if validation.missing_configs:
            logger.warning(f"Missing configs: {validation.missing_configs}")
        if validation.unresolvable_builders:
            logger.warning(f"Unresolvable builders: {validation.unresolvable_builders}")
        if validation.config_errors:
            logger.warning(f"Config errors: {validation.config_errors}")

### Put it Together: Pipeline Generation from DAG

In [59]:
# Convert DAG to pipeline and get report
try:
    logger.info(f"Converting DAG to pipeline")
    template_pipeline, report = dag_compiler.compile_with_report(
        dag=dag
    )
        
    # Log report summary
    logger.info(f"Conversion complete: {report.summary()}")
    for node, details in report.resolution_details.items():
        logger.info(f"  {node} → {details['config_type']} ({details['builder_type']})")
        
    # Log pipeline creation details
    logger.info(f"Pipeline '{template_pipeline.name}' created successfully")
    logger.info(f"Pipeline ARN: {template_pipeline.arn if hasattr(template_pipeline, 'arn') else 'Not available until upserted'}")
    logger.info("To upsert the pipeline, call pipeline.upsert()")       
except Exception as e:
    logger.error(f"Failed to convert DAG to pipeline: {e}")
    raise

2025-11-08 05:08:13,463 - INFO - Converting DAG to pipeline
2025-11-08 05:08:13,463 - INFO - Compiling DAG with detailed reporting
2025-11-08 05:08:13,464 - INFO - Compiling DAG with 4 nodes to pipeline
2025-11-08 05:08:13,464 - INFO - Creating template for DAG with 4 nodes
2025-11-08 05:08:13,464 - WARNING - Could not import config_class_detector, using fallback implementation
2025-11-08 05:08:13,465 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-08 05:08:13,465 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-11-08 05:08:13,466 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-11-08 05:08:13,466 - INFO - ✅ Registry info loaded: 34 steps
2025-11-08 05:08:13,467 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-11-08 05:08:13,467 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMake

### Pipeline Template

After the pipeline is generated, we can retrieve the pipeline template

In [60]:
pipeline_template_builder = dag_compiler.get_last_template()

## Start Execution

In [61]:
role_arn = pipeline_session.get_caller_identity_arn()
role_arn

'arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default'

In [62]:
pipeline_description=PIPELINE_DESCRIPTION

In [63]:
PIPELINE_DESCRIPTION

'BuyerAbuseRnR pytorch Model NA'

### Upsert

In [64]:
template_pipeline.upsert(
                role_arn=role_arn, description=pipeline_description
            )

2025-11-08 05:08:14,243 - WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
2025-11-08 05:08:14,606 - WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
2025-11-08 05:08:14,652 - WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
2025-11-08 05:08:14,713 - WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDe

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:178936618742:pipeline/lukexie-BuyerAbuseRnR-pytorch-NA-0-0-1-pipeline',
 'PipelineVersionId': 39,
 'ResponseMetadata': {'RequestId': 'edb8c0ac-7263-4a82-9688-814c4413198d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'edb8c0ac-7263-4a82-9688-814c4413198d',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '138',
   'date': 'Sat, 08 Nov 2025 05:08:15 GMT'},
  'RetryAttempts': 0}}

### Start

In [65]:
pipeline_execution_parameters={
    "EXECUTION_S3_PREFIX": f"s3://{bucket_name}/pipeline/{PIPELINE_NAME}/{execution_id}",
    "KMS_ENCRYPTION_KEY_PARAM": kms_key_id,
    "VPC_SUBNET": vpc_subnet_id,
    "SECURITY_GROUP_ID": security_group_id,
}

In [66]:
pipeline_execution = template_pipeline.start(
                parameters=pipeline_execution_parameters
            )